In [1]:
%matplotlib widget

In [2]:
%load_ext autoreload
%autoreload 2

In [48]:
import torch
import torch.nn as nn
from neural_network.cam.cams import SmoothGradCAMpp, SSCAM, ScoreCAM
from neural_network.trainer import Agent
import matplotlib.pyplot as plt
from neural_network.utils.utils import interactive_slices,interactive_slices_masked
from skimage.transform import resize

In [4]:
agent = Agent('resnet50');

Setup configurations...
Dataset sizes - Training: 421 Validation: 47 Test: 0
architecture [ResNet] was created
model [LightningModel] was created


GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.


In [6]:
agent.model;

LightningModel(
  (model): ResNet(
    (conv1): Conv3d(1, 64, kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
    (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (bn3): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (

In [7]:
val_iter = iter(agent.dataset.val_dataloader())

In [8]:
first_image, first_label = next(val_iter)

In [9]:
# Import M3d-CAM
from medcam import medcam

# Init your model and dataloader
model = agent.model
data_loader = agent.dataset.val_dataloader()

# Inject model with M3d-CAM
model = medcam.inject(model, output_dir="attention_maps", layer='model.layer4',save_maps=True,data_shape=(79,224,224))

# Continue to do what you're doing...
# In this case inference on some new data
model.eval()
output = model(first_image[0].unsqueeze(0))


In [10]:
from skimage.transform import resize
from matplotlib import cm
def overlay(raw_input, attention_map):
    if isinstance(raw_input, torch.Tensor):
        raw_input = raw_input.detach().cpu().numpy()
        if raw_input.shape[0] == 1 or raw_input.shape[0] == 3:
            raw_input = raw_input.permute(1,2,0)
    if np.max(raw_input) > 1:
        raw_input = raw_input.astype(np.float)
        raw_input /= 255
    print(raw_input.shape)
    attention_map = resize(attention_map.permute(1,2,0), raw_input.shape)
    attention_map = cm.jet_r(attention_map)
    attention_map = (attention_map.astype(np.float) + raw_input.astype(np.float)) / 2
    attention_map *= 255
    return attention_map

darp = medcam.medcam_utils.normalize(model.medcam_dict['current_attention_map'])
darp = resize(darp.squeeze(0).squeeze(0),(79,224,224))

In [80]:
import numpy as np
fig = plt.figure()
plt.imshow((first_image[0].squeeze(0)[50] * 255).numpy().astype(np.uint8), cmap='gray')
plt.imshow((darp[50] * 255).astype(np.uint8), cmap='jet', alpha=0.3)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [81]:
from neural_network.utils.utils import interactive_slices
testplot = interactive_slices()
testplot.multi_slice_viewer(darp)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [88]:
asd = interactive_slices_masked()
asd.multi_slice_viewer(first_image[5],darp)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [90]:
asd.cycle(0.1)

KeyboardInterrupt: 